In [1]:
import math
import pandas as pd
import re
import numpy as np
import itertools
import collections
from collections import defaultdict, Counter
import string
from sklearn.preprocessing import LabelEncoder

In [2]:
ls

 Volume in drive C is Acer
 Volume Serial Number is 3818-7542

 Directory of C:\Users\phvpa\Desktop\Experimants\DataSet

30-10-2021  21:17    <DIR>          .
30-10-2021  12:45    <DIR>          ..
30-10-2021  21:11    <DIR>          .ipynb_checkpoints
30-10-2021  16:09            42,127 Kan_Preprocessing _and_spliting_2750and250.ipynb
30-10-2021  12:45    <DIR>          Kannada
30-10-2021  16:09            38,450 Mal_Preprocessing _and_spliting_5000and500.ipynb
30-10-2021  12:45    <DIR>          Malyalam
30-10-2021  12:54    <DIR>          NumpyFiles
30-10-2021  16:25    <DIR>          Tamil
30-10-2021  21:17            43,071 Tamil_Preprocessing _and_spliting_1600and150.ipynb
               3 File(s)        123,648 bytes
               7 Dir(s)  316,347,539,456 bytes free


In [3]:
#pd.options.display.max_rows = None

In [4]:
train_df = pd.read_csv("Tamil/tamil_sentiment_full_train.tsv", sep='\t')
print(len(train_df))

#test_df = pd.read_csv("Mal_sentiment_full_dev.tsv", header=None, sep='\t')
valid_df = pd.read_csv("Tamil/tamil_sentiment_full_dev.tsv", sep='\t')
print(len(valid_df))

test_df = pd.read_csv("Tamil/tamil_sentiment_full_test_withoutlabels.tsv", sep='\t')
len(test_df)

35656
3962


4402

In [5]:
train_df

,text,category
0,Vani bhojam fans hit like solli 500 like Vangi...,unknown_state
1,I love you ajith very I like,Positive
2,ennaya trailer Ku mudi Ellam nikkudhu... Vera ...,Positive
3,Vijay Annaa Ur Maassssss Therrrrriiiiii,Positive
4,நம்ப நடே நாசாமா தான் போச்சி,Negative
...,...,...
35651,ஒருவர் செய்த தவறுக்காக எல்லாரையும் பழி போடுவது...,Mixed_feelings
35652,Repeated Mode to Watching Theri Trailer & Ther...,Positive
35653,yeevanikuachu pula kutti erudhal oodi poyidu....,Negative
35654,She is looking like laughing budha,Positive


In [6]:
train_df['category']

0         unknown_state
1              Positive
2              Positive
3              Positive
4              Negative
              ...      
35651    Mixed_feelings
35652          Positive
35653          Negative
35654          Positive
35655          Positive
Name: category, Length: 35656, dtype: object

In [7]:
#removing all the emoji from the given data

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


for i in range(len(train_df)):
    temp1 = deEmojify(train_df['text'][i])
    train_df['text'][i] = temp1

for j in range(len(valid_df)):
    temp2 = deEmojify(valid_df['text'][j])
    valid_df['text'][j] = temp2

for k in range(len(test_df)):
    temp3 = deEmojify(test_df['text'][k])
    test_df['text'][k]=temp3

In [8]:
def preprocess_data(text):
  user_name = '@[A-Za-z0-9]+' # Pattern for matching the user names in the tweet
  has_tag = '#[A-Za-z0-9]+' # Pattern for matching the hasgtags in the tweer
  url = 'https?:\/\/[A-Za-z0-9\.\/\-]+' # Pattern for matching the URLs in the tweet
  #emoji_pattern = r'/[x{1F600}-x{1F64F}]/u'
  
  x = text.lower()  # Lower casing all the characters
  x = re.sub(user_name, '', x) # Replace the username with an empty characrer
  x = re.sub(has_tag,'',x) # Replace the hashtags with an empty characrer
  x = re.sub(url, '', x) # Replace the URLs with an empty characrer
  #x = re.sub(emoji_pattern, '', x)
  x = x.translate(str.maketrans('', '', string.punctuation)) # Replace all the characters except alphabets and digits from the tweet

  return x

In [9]:
train_df['text'] = train_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
valid_df['text'] = valid_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
test_df['text'] = test_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe

In [10]:
print(Counter(train_df['category']))

Counter({'Positive': 20069, 'unknown_state': 5628, 'Negative': 4271, 'Mixed_feelings': 4020, 'not-Tamil': 1667, 'Positive ': 1})


In [11]:
for i in range(len(train_df)):
    if train_df['category'][i] == 'Positive ':
        train_df['category'][i] = 'Positive'

In [12]:
np.save('NumpyFiles\Tamil\Fulldata\Tamil_Train_Full_data.npy',list(train_df['text']))
np.save('NumpyFiles\Tamil\Fulldata\Tamil_Train_Full_labels.npy',list(train_df['category']))
np.save('NumpyFiles\Tamil\Fulldata\Tamil_Val_Full_data.npy',list(valid_df['text']))
np.save('NumpyFiles\Tamil\Fulldata\Tamil_Val_Full_labels.npy',list(valid_df['category']))
np.save('NumpyFiles\Tamil\Fulldata\Tamil_Test_Full_data.npy',list(valid_df['text']))

In [13]:
temp1 = np.load('NumpyFiles\Tamil\Fulldata\Tamil_Train_Full_data.npy')
temp2 = np.load('NumpyFiles\Tamil\Fulldata\Tamil_Train_Full_labels.npy')
temp3 = np.load('NumpyFiles\Tamil\Fulldata\Tamil_Val_Full_data.npy')
temp4 = np.load('NumpyFiles\Tamil\Fulldata\Tamil_Val_Full_labels.npy')
temp5 = np.load('NumpyFiles\Tamil\Fulldata\Tamil_Test_Full_data.npy')

In [14]:
temp1.shape, temp2.shape, temp3.shape, temp4.shape

((35656,), (35656,), (3962,), (3962,))

In [15]:
train_df['category'][0], len(train_df)

('unknown_state', 35656)

# Dividing train and valid separatly:

### Train data

In [16]:
df_temp2 = train_df.sort_values('category')
df_temp2

,text,category
11296,abhi se advance booking suru ho jaye kon kon...,Mixed_feelings
3436,0108 semma rangaraj pandey mass sir rangaraj p...,Mixed_feelings
22494,viswasam trailera vida nalla iruku vera level hit,Mixed_feelings
31235,vazhkayile thaniya ninnu jaikiravean yaar ha i...,Mixed_feelings
11770,shankar innu pannirukalam nu sonnavanlam go b...,Mixed_feelings
...,...,...
20394,046 kathii trailer feeeel adichavange like,unknown_state
20399,oho 96la ilayaraja paatu vantha ippo irrukura ...,unknown_state
20405,plz dubbed in kannada also,unknown_state
20416,bigil trailer wait pandravanga like here,unknown_state


In [17]:
train = Counter(train_df['category'])
train_sort = dict(sorted(train.items(), key=lambda item: item[0]))

print('Traindf: \n',train_sort,'\n')
temp = 0
for key in train_sort:
    print(key,'starts from: ', temp)
    temp = temp + int(train_sort[key])
print(key,'ends at: ', temp)

Traindf: 
 {'Mixed_feelings': 4020, 'Negative': 4271, 'Positive': 20070, 'not-Tamil': 1667, 'unknown_state': 5628} 

Mixed_feelings starts from:  0
Negative starts from:  4020
Positive starts from:  8291
not-Tamil starts from:  28361
unknown_state starts from:  30028
unknown_state ends at:  35656


In [18]:
#print(df_temp2['category'].tolist())

In [19]:
count2 = 1600
train_Mixed_feelings = df_temp2[:count2]
train_Negative = df_temp2[4025:4025+count2]
train_Positive=df_temp2[8295:8295+count2]
train_not_Tamil = df_temp2[28370:28370+count2]
train_unknown_state = df_temp2[30035:30035+count2]

In [20]:
print('total:',count2*5)
len(train_Mixed_feelings), len(train_Negative), len(train_Positive), len(train_not_Tamil), len(train_unknown_state)

total: 8000


(1600, 1600, 1600, 1600, 1600)

In [21]:
Train_frames = [train_Mixed_feelings, train_Negative, train_Positive, train_not_Tamil, train_unknown_state ]
Train_results = pd.concat(Train_frames)
Train_results.shape

(8000, 2)

In [22]:
# Train_results

In [23]:
Counter(Train_results['category'])

Counter({'Mixed_feelings': 1600,
         'Negative': 1600,
         'Positive': 1600,
         'not-Tamil': 1600,
         'unknown_state': 1600})

In [24]:
Sub_Train = Train_results.sample(frac = 1)

In [25]:
Sub_Train_list = Sub_Train['text'].tolist()
Sub_Train_labels_list = Sub_Train['category'].tolist()

### Val Data

In [26]:
df_temp = valid_df.sort_values('category')
df_temp.shape

(3962, 2)

In [27]:
val = Counter(valid_df['category'])
val_sort = dict(sorted(val.items(), key=lambda item: item[0]))

print('\nvaliddf:\n',val_sort,'\n')
temp = 0
for key in val_sort:
    print(key,'starts from: ', temp)
    temp = temp + int(val_sort[key])

print(key,'ends at: ', temp)


validdf:
 {'Mixed_feelings': 438, 'Negative': 480, 'Positive': 2257, 'not-Tamil': 176, 'unknown_state': 611} 

Mixed_feelings starts from:  0
Negative starts from:  438
Positive starts from:  918
not-Tamil starts from:  3175
unknown_state starts from:  3351
unknown_state ends at:  3962


In [28]:
count = 150
val_Mixed_feelings = df_temp[:count]
val_Negative = df_temp[440:440+count]
val_Positive=df_temp[920:920+count]
val_not_Tamil = df_temp[3180:3180+count]
val_unknown_state = df_temp[3355:3355+count]

In [29]:
print('total:',count*5)
len(val_Mixed_feelings), len(val_Negative), len(val_Positive), len(val_not_Tamil), len(val_unknown_state)

total: 750


(150, 150, 150, 150, 150)

In [30]:
val_frames = [val_Mixed_feelings, val_Negative, val_Positive, val_not_Tamil, val_unknown_state ]

val_result = pd.concat(val_frames) #concat

In [31]:
Counter(val_result['category'])

Counter({'Mixed_feelings': 150,
         'Negative': 150,
         'Positive': 150,
         'not-Tamil': 150,
         'unknown_state': 150})

In [32]:
Sub_val = val_result.sample(frac = 1) #shuffle

In [33]:
Sub_val_list = Sub_val['text'].tolist()
Sub_val_labels_list = Sub_val['category'].tolist()

# Saving Choosen data into numpy files

In [34]:
#"C:\Users\phvpa\Desktop\Experimants\DataSet\NumpyFiles\Kan\Split2750and250"
Tamil_train = np.array(Sub_Train_list)
np.save('NumpyFiles\Tamil\Split1600and150\Tamil_Train_data_1600.npy',Tamil_train)
Train_train_label = np.array(Sub_Train_labels_list)
np.save('NumpyFiles\Tamil\Split1600and150\Tamil_Train_label_1600.npy',Train_train_label)

In [35]:
temp21 =np.load('NumpyFiles\Tamil\Split1600and150\Tamil_Train_data_1600.npy')
print(temp21.shape)
temp22 =np.load('NumpyFiles\Tamil\Split1600and150\Tamil_Train_label_1600.npy')
print(temp22.shape)

(8000,)
(8000,)


In [36]:
Tamil_val = np.array(Sub_val_list)
np.save("NumpyFiles\Tamil\Split1600and150\Tamil_Val_data_150.npy",Tamil_val)
Tamil_val_label = np.array(Sub_val_labels_list)
np.save("NumpyFiles\Tamil\Split1600and150\Tamil_val_label_150.npy",Tamil_val_label)

In [37]:
temp11 =np.load('NumpyFiles\Tamil\Split1600and150\Tamil_Val_data_150.npy')
print(temp11.shape)
temp12 =np.load('NumpyFiles\Tamil\Split1600and150\Tamil_val_label_150.npy')
print(temp12.shape)

(750,)
(750,)


In [38]:
Traindf = pd.DataFrame({'Data' : temp21,'label' : temp22}, columns=['Data', 'label'])
Traindf

,Data,label
0,vivegam trailer mersal all d best thala team,Mixed_feelings
1,any malayalies rajani sir hit like here,unknown_state
2,ரஜினி சாரை பத்தி பேசினா மட்டும்தான் உங்க படத்த...,unknown_state
3,romba naal kalichu oru naal padam politics bas...,unknown_state
4,இப்படம் வெற்றி பெற மனிதர்கள் சார்பாக வாழ்த்துக...,Positive
...,...,...
7995,anils fans ellaarum vaya mudikitu odungada ell...,Mixed_feelings
7996,ஜாதி படம் எடுத்தா ஓடும் என்ற ஆசை உருவானாதா அடு...,Mixed_feelings
7997,தேர்தலில் இந்த ஒற்றுமை இருந்தால் சிலபேருக்கு ச...,Mixed_feelings
7998,okkkaaa makkkaaaa mass pppppaaaaaaaa this is w...,Positive


In [39]:
Testdf = pd.DataFrame({'Data' : temp11,'label' : temp12}, columns=['Data', 'label'])
Testdf

,Data,label
0,nallave illa ipd over ah panni panni tha ippo...,Negative
1,iam waiting for kaappaan i love you surya anna,not-Tamil
2,suyama sinthika therinjavan tha super hero,Mixed_feelings
3,sai pallavi ni evado baga thinesthunnadu papa ...,unknown_state
4,srilankan bakery van bgm,Negative
...,...,...
745,enum nala pani irukalaam orginal pink trailer ...,Positive
746,aik aur movie ary yar bas karoo tum loog takty...,not-Tamil
747,என்னைக்குமே தலைவர் தான் மாஸ்,Positive
748,என்ன ஜி படம் ஷங்கர் சார் range la எடுகிறீங்களa...,Mixed_feelings
